In [4]:
!pip install opencv-python
!pip install scikit-learn
!pip install flask



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/ae/20/6d1a0a61d468b37a142fd90bb93c73bc1c2205db4a69ac630ed218c31612/scikit_learn-1.5.0-cp311-cp311-win_amd64.whl.metadata
  Using cached scikit_learn-1.5.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for scipy>=1.6.0 from https://files.pythonhosted.org/packages/4a/48/4513a1a5623a23e95f94abd675ed91cfb19989c58e9f6f7d03990f6caf3d/scipy-1.13.1-cp311-cp311-win_amd64.whl.metadata
  Using cached scipy-1.13.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Obtaining dependency information for joblib>=1.2.0 from https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl.metadata
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf7a134b9ab11a67b0cf0726453cedd9c5043a4fe7


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split


data_dir = "dataset_full"


images = []
labels = []


class_names = os.listdir(data_dir)
print(class_names)
class_indices = {class_name: idx for idx, class_name in enumerate(class_names)}


for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    file_names = os.listdir(class_dir)[1:] 
    for file_name in file_names:
        file_path = os.path.join(class_dir, file_name)
        image = cv2.imread(file_path)
        image = cv2.resize(image, (128, 128)) 
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
        images.append(image)
        labels.append(class_indices[class_name])


images = np.array(images, dtype=np.uint8)
labels = np.array(labels)


images = images / 255.0


X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

print("Data loaded and preprocessed.")


['Building', 'Forest', 'Glacier', 'Mountains', 'Sea', 'Streets']
Data loaded and preprocessed.


In [7]:
import cv2
import numpy as np



def histogram_equalization(image):
    """Apply histogram equalization to the image."""

    if image.dtype != np.uint8:
        image = (image * 255).astype(np.uint8)

    return cv2.equalizeHist(image)

def edge_detection_canny(image, low_threshold=50, high_threshold=150):
    """Apply Canny edge detection."""
    image = (image * 255).astype(np.uint8) 
    return cv2.Canny(image, low_threshold, high_threshold)

def gaussian_blur(image, kernel_size=(5, 5)):
    """Apply Gaussian blur to the image."""
    return cv2.GaussianBlur(image, kernel_size, 0)



def extract_features(images):
    features = []
    for image in images:
   
        hist_eq = histogram_equalization(image)
        gauss_blur = gaussian_blur(image)
        canny_edges = edge_detection_canny(image)
        

        feature_vector = np.concatenate([
            hist_eq.flatten(),
            gauss_blur.flatten(),
            canny_edges.flatten()
        ])
        
        features.append(feature_vector)
    
    return np.array(features)


X_train_features = extract_features(X_train)
X_val_features = extract_features(X_val)

print("Feature extraction completed.")


Feature extraction completed.


In [8]:
from sklearn.decomposition import PCA


def apply_pca(features, n_components=100):
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(features)
    return reduced_features, pca

X_train_reduced, pca = apply_pca(X_train_features)
X_val_reduced = pca.transform(X_val_features)

print("Dimensionality reduction completed.")


Dimensionality reduction completed.


In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report


def train_knn(X_train, y_train, n_neighbors=5):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    knn.fit(X_train, y_train)
    return knn

knn_model = train_knn(X_train_reduced, y_train)


y_val_pred = knn_model.predict(X_val_reduced)
accuracy = accuracy_score(y_val, y_val_pred)

print(f"Validation Accuracy: {accuracy}")
print(classification_report(y_val, y_val_pred, target_names=class_names))


Validation Accuracy: 0.6854146806482364
              precision    recall  f1-score   support

    Building       0.46      0.41      0.43       112
      Forest       0.82      0.97      0.89       534
     Glacier       0.49      0.37      0.42       101
   Mountains       0.45      0.67      0.54       105
         Sea       0.33      0.16      0.21        90
     Streets       0.78      0.33      0.46       107

    accuracy                           0.69      1049
   macro avg       0.55      0.48      0.49      1049
weighted avg       0.67      0.69      0.66      1049



In [10]:
import pickle

with open('knn_model.pkl', 'wb') as f:
    pickle.dump(knn_model, f)

with open('pca.pkl', 'wb') as f:
    pickle.dump(pca, f)
